## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Cockburn Town,21.4612,-71.1419,78.76,65,40,11.50,TC,scattered clouds
1,1,Avarua,-21.2078,-159.7750,82.45,78,40,10.36,CK,scattered clouds
2,2,Tahe,52.3333,124.7333,15.19,93,97,3.65,CN,overcast clouds
3,3,Mataura,-46.1927,168.8643,61.32,69,4,2.46,NZ,clear sky
4,4,Hobart,-42.8794,147.3294,64.26,74,0,8.05,AU,heavy intensity rain


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Cockburn Town,21.4612,-71.1419,78.76,65,40,11.50,TC,scattered clouds
1,1,Avarua,-21.2078,-159.7750,82.45,78,40,10.36,CK,scattered clouds
13,13,Cabo San Lucas,22.8909,-109.9124,86.61,14,75,9.22,MX,broken clouds
17,17,Kununurra,-15.7667,128.7333,75.18,100,98,3.44,AU,overcast clouds
23,23,Puerto Ayora,-0.7393,-90.3518,84.16,76,96,12.24,EC,light rain
25,25,Hilo,19.7297,-155.0900,80.33,59,0,10.36,US,clear sky
26,26,Kapaa,22.0752,-159.3190,79.48,65,20,8.05,US,few clouds
27,27,Hobyo,5.3505,48.5268,80.31,63,100,12.95,SO,overcast clouds
28,28,Arraial Do Cabo,-22.9661,-42.0278,78.66,89,40,5.75,BR,moderate rain
29,29,Ciudad Bolivar,8.1222,-63.5497,89.35,46,75,17.13,VE,broken clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                204
City                   204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Country                204
Current Description    204
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cockburn Town,TC,78.76,scattered clouds,21.4612,-71.1419,
1,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
13,Cabo San Lucas,MX,86.61,broken clouds,22.8909,-109.9124,
17,Kununurra,AU,75.18,overcast clouds,-15.7667,128.7333,
23,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,
25,Hilo,US,80.33,clear sky,19.7297,-155.0900,
26,Kapaa,US,79.48,few clouds,22.0752,-159.3190,
27,Hobyo,SO,80.31,overcast clouds,5.3505,48.5268,
28,Arraial Do Cabo,BR,78.66,moderate rain,-22.9661,-42.0278,
29,Ciudad Bolivar,VE,89.35,broken clouds,8.1222,-63.5497,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Skip")

Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cockburn Town,TC,78.76,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
1,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
13,Cabo San Lucas,MX,86.61,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
17,Kununurra,AU,75.18,overcast clouds,-15.7667,128.7333,Hotel Kununurra
23,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
25,Hilo,US,80.33,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
26,Kapaa,US,79.48,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
27,Hobyo,SO,80.31,overcast clouds,5.3505,48.5268,Iftin hotel hobyo
28,Arraial Do Cabo,BR,78.66,moderate rain,-22.9661,-42.0278,Pousada Porto Praia
29,Ciudad Bolivar,VE,89.35,broken clouds,8.1222,-63.5497,Hotel Edi


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

In [24]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))